# Project 3 - Data Delivery, Backend, Visualization
## The Data extraction and Transformation Module

In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
import us
from us import states
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2021)

## Connect to Mongo DB

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.CensusData
collection = db.items

## Web scraping

* Call URL and gather response and use BeautifulSoup to parse the data
* Reference - Module 12 / Chapter 2 / Lesson 5 Mongo Scraping

### Get data

In [15]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_counties_by_U.S._state_and_territory'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')
years=[2018,2019,2020]
complete_years=pd.DataFrame()
for curr_year in years:
    census_data = c.acs5.state_county(fields = ('NAME', "B01003_001E", "B01002_001E", "B19301_001E", "B17001_002E"),
                                      state_fips = "*",
                                      county_fips = "*", year = curr_year)

    census_data_curr_year=pd.DataFrame(census_data)
    # Column Reordering
    census_data_curr_year.rename(columns={"B01003_001E": "Population",
                                     "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                     "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name"},inplace=True)

    census_data_curr_year["Year"]=curr_year
    frames=[complete_years,census_data_curr_year]
    complete_years=pd.concat(frames)
    #complete_years.merge(census_data_curr_year, how='left', on=["state","county","Year"])
    #census_data_curr_year.head()
complete_years.head()

Name  Population  Median Age  Per Capita Income  \
0  Sedgwick County, Kansas    512064.0        35.2            28673.0   
1  Republic County, Kansas      4686.0        51.1            28901.0   
2    Graham County, Kansas      2545.0        51.9            26685.0   
3   Douglas County, Kansas    119319.0        29.5            30315.0   
4  Sheridan County, Kansas      2506.0        44.3            31526.0   

   Poverty Count state county  Year  
0        70537.0    20    173  2018  
1          490.0    20    157  2018  
2          235.0    20    065  2018  
3        20749.0    20    045  2018  
4          126.0    20    179  2018

In [19]:
complete_years.reset_index()

index                         Name  Population  Median Age  \
0         0      Sedgwick County, Kansas    512064.0        35.2   
1         1      Republic County, Kansas      4686.0        51.1   
2         2        Graham County, Kansas      2545.0        51.9   
3         3       Douglas County, Kansas    119319.0        29.5   
4         4      Sheridan County, Kansas      2506.0        44.3   
...     ...                          ...         ...         ...   
9656   3216   Renville County, Minnesota     14572.0        44.0   
9657   3217     Roseau County, Minnesota     15259.0        41.6   
9658   3218  Sherburne County, Minnesota     96015.0        36.1   
9659   3219     Steele County, Minnesota     36710.0        39.2   
9660   3220       Todd County, Minnesota     24603.0        43.9   

      Per Capita Income  Poverty Count state county  Year  
0               28673.0        70537.0    20    173  2018  
1               28901.0          490.0    20    157  2018  
2               26685.0          235.0    20    065  2018  
3               30315.0        20749.0    20    045  2018  
4               31526.0          126.0    20    179  2018  
...                 ...            ...   ...    ...   ...  
9656            31243.0         1373.0    27    129  2020  
9657            31452.0         1133.0    27    135  2020  
9658            36022.0         4953.0    27    141  2020  
9659            34648.0         2887.0    27    147  2020  
9660            26427.0         3276.0    27    153  2020  

[9661 rows x 9 columns]

In [21]:
complete_years.to_json("complete_years.json",orient="records")

### Parse Data

In [ ]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('div', class_='div-col')

county_info_df = pd.DataFrame()

# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        county_info = result.find('a').text

        county = county_info.split(',')

        # Run only if title, price, and link are available
        if (county):
            # # Print results
            # print('-------------')
            # print(title)
            # print(price)
            # print(link)
            # Dictionary to be inserted as a MongoDB document
            county_info = {
                'County': county[0],
                'State': county[1]
            }
            pd.
    except Exception as e:
        print(e)


collection.insert_one(post)